In [42]:
f = open("data/train.txt",'r')
train = f.readlines()
f.close()

f = open("data/valid.txt",'r')
valid = f.readlines()
f.close()

f = open("data/test.txt",'r')
test = f.readlines()
f.close()

word_vocab = {'\n':2, ' ':1, '':0}
char_vocab = {'\n':2, ' ':1, '':0}

In [188]:
leng = 0
for i in train :
    cnt = 0
    words = i.split()
    for word in words :
        for j in word :
            cnt+=1
    if leng < cnt :
        leng = cnt
print(leng)

439


In [260]:
word_id = 3
char_id = 3

train_word_data = []
train_word_label = []
train_char_data = []
#train_char_data = np.zeros((1, 520))
valid_word_data = []
valid_char_data = []
test_word_data = []
test_char_data = np.zeros((1, 439))
cnt = 0
for i in train :
    data_word = []
    label_word = []
    data_char = []
    words = i.split()
    for word in words :
        if word not in word_vocab :
            word_vocab[word] = word_id
            word_id+=1
        for j in word :
            if j not in char_vocab :
                char_vocab[j] = char_id
                char_id += 1 
            data_char.append(j)
        data_word.append(word)
        label_word.append(word_vocab[word])
        #data_word.append(1)
        #data_char.append(1) #space 
    #data_word.append(2) #\n
    #data_char.append(2)

    """if len(data_char) < 520 :
        j = len(data_char)
        for i in range(j, 520) :
            data_char.append(0)"""
    
    #train_char_data[cnt] = (torch.FloatTensor(data_char))
    cnt+=1
    train_char_data.append(data_char)
    train_word_data.append(data_word)
    train_word_label.append(label_word)

In [45]:
for i in valid :
    words = i.split()
    for word in words :
        if word not in word_vocab :
            word_vocab[word] = word_id
            word_id+=1
        for j in word :
            if j not in char_vocab :
                char_vocab[j] = char_id
                char_id += 1  
            valid_char_data.append(char_vocab[j])
        valid_word_data.append(word_vocab[word])
        valid_word_data.append(1)
        valid_char_data.append(1) #space 
    valid_word_data.append(0) #\n
    valid_char_data.append(0)

In [155]:
for i in test :
    words = i.split()
    for word in words :
        if word not in word_vocab :
            word_vocab[word] = word_id
            word_occur+=1
        for j in word :
            if j not in char_vocab :
                char_vocab[j] = char_id
                char_id= 1  
            test_char_data.append(char_vocab[j])
        test_word_data.append(word_vocab[word])
        test_word_data.append(1)
        test_char_data.append(1) #space 
    test_word_data.append(0) #\n
    test_char_data.append(0)

In [236]:
train_char_data

tensor([[ 2.,  3.,  4.,  ...,  0.,  0.,  0.],
        [25., 11.,  3.,  ...,  0.,  0.,  0.],
        [19.,  4., 32.,  ...,  0.,  0.,  0.],
        ...,
        [ 5.,  3., 15.,  ...,  0.,  0.,  0.],
        [11.,  9.,  1.,  ...,  0.,  0.,  0.],
        [11.,  6.,  1.,  ...,  0.,  0.,  0.]])

In [261]:
data = {}
data['train_char'] = train_char_data
data['train_word'] = train_word_data
data['valid_char'] = np.array(valid_char_data)
data['valid_word'] = np.array(valid_word_data)
data['test_char'] = np.array(test_char_data)
data['test_word'] = np.array(test_word_data)
data['word_vocab'] = word_vocab
data['char_vocab'] = char_vocab

In [238]:
data['train_char'].shape

torch.Size([42068, 520])

In [116]:
data['train_char'].reshape(439, -1).shape

ValueError: cannot reshape array of size 42068 into shape (439,newaxis)

In [262]:
import pickle

with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

In [215]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [218]:
with open('data.pickle', 'rb') as f :
    data = pickle.load(f)

train_input = data['train_char']
train_label = data['train_char']
valid_input = torch.from_numpy(data['valid_char'])
valid_label = torch.from_numpy(data['valid_char'])

In [271]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return torch.autograd.Variable(tensor)

In [277]:
sentence_in = prepare_sequence(train_input[1], char_vocab)

In [278]:
sentence_in.size()

torch.Size([64])